<a href="https://colab.research.google.com/github/laurenneal/capstone-visual-neuroscience/blob/main/Feature_Engineering_from_Raw_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import sys
from numpy.ma.core import ceil, floor
from more_itertools import sliced

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! ls drive/MyDrive/DS6011_Capstone_VisualNeuroscience/Seeded_CNMF/Extracted_Features/

210728_0_20220304T003321RAW_stimulus_data.csv
210728_0_20220304T003321_sparse_stim1_10f.csv
210728_0_20220304T003321_sparse_stim1_15f.csv
210728_0_20220304T003321_sparse_stim1_20f.csv
210728_0_20220304T003321_sparse_stim1_5f.csv
210731_0_20220304T005413RAW_stimulus_data.csv
210731_0_20220304T005413_sparse_stim1_10f.csv
210731_0_20220304T005413_sparse_stim1_15f.csv
210731_0_20220304T005413_sparse_stim1_20f.csv
210731_0_20220304T005413_sparse_stim1_5f.csv
210815_0__20220213T070259RAW_stimulus_data.csv
210815_0_20220213T070259_sparse_stim1_10f.csv
210815_0_20220213T070259_sparse_stim1_15f.csv
210815_0_20220213T070259_sparse_stim1_20f.csv
210815_0_20220213T070259_sparse_stim1_5f.csv
210815_0_allStacks_mc_mix1_syt_rawExtracts
210816_0_20220304T112124RAW_stimulus_data.csv
210816_0_20220304T112124_sparse_stim1_10f.csv
210816_0_20220304T112124_sparse_stim1_15f.csv
210816_0_20220304T112124_sparse_stim1_20f.csv
210816_0_20220304T112124_sparse_stim1_5f.csv
210816_1_20220304T113821RAW_stimulus_dat

## Functions
This section contains functions that can be used on datasets for feature engineering

In [4]:
raw_df = pd.read_csv('drive/MyDrive/DS6011_Capstone_VisualNeuroscience/Seeded_CNMF/Extracted_Features/210815_0__20220213T070259RAW_stimulus_data.csv', index_col=['stack','label','roi','frame']).drop(columns='Unnamed: 0')
raw_df

filename  \
stack label roi frame                                                      
1     tm2   1   0      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...   
                1      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...   
                2      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...   
                3      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...   
                4      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...   
...                                                                  ...   
10    tm4   15  5508   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5509   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5510   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5511   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5512   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   

                           resp     stim1       stim2     stim3  
stack label roi frame                                            
1     tm2   1   0     -0.106575 -0.395877  160.447960  0.110063  
                1      0.244535 -0.395877    0.000000  0.000000  
                2      0.398458 -0.395877    0.000000  0.000000  
                3      0.619290 -0.138699  170.252676  0.030790  
                4      0.627615  0.890013  -62.485104  0.122134  
...                         ...       ...         ...       ...  
10    tm4   15  5508   0.578604  0.113874    0.000000  0.000000  
                5509   0.417371 -0.289958  -96.522646  0.089182  
                5510   0.324171 -0.370724  -89.799347  0.019588  
                5511   0.255033 -0.370724    0.000000  0.000000  
                5512   0.121880 -0.370724    0.000000  0.000000  

[826950 rows x 5 columns]

## REMOVE THE OUTLIER ROI

In [5]:
#access the 11th roi in the first stack and drop all frames for that roi
raw_df = raw_df.drop(raw_df.loc[1,:,11].index)

In [6]:
raw_df

filename  \
stack label roi frame                                                      
1     tm2   1   0      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...   
                1      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...   
                2      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...   
                3      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...   
                4      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...   
...                                                                  ...   
10    tm4   15  5508   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5509   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5510   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5511   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5512   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   

                           resp     stim1       stim2     stim3  
stack label roi frame                                            
1     tm2   1   0     -0.106575 -0.395877  160.447960  0.110063  
                1      0.244535 -0.395877    0.000000  0.000000  
                2      0.398458 -0.395877    0.000000  0.000000  
                3      0.619290 -0.138699  170.252676  0.030790  
                4      0.627615  0.890013  -62.485104  0.122134  
...                         ...       ...         ...       ...  
10    tm4   15  5508   0.578604  0.113874    0.000000  0.000000  
                5509   0.417371 -0.289958  -96.522646  0.089182  
                5510   0.324171 -0.370724  -89.799347  0.019588  
                5511   0.255033 -0.370724    0.000000  0.000000  
                5512   0.121880 -0.370724    0.000000  0.000000  

[821437 rows x 5 columns]

## Train Test Split on Stacks

In [7]:
def trainTestSplit(df, splt = [.7,.2,.1]):
  # splits on roi, rois accross stacks will have the same group (important if stacks are from the same video)
  # split is the train test valuidation split (proportions given in that order)
  if sum(splt) < 0.9999 or sum(splt) > 1.0001:
    sys.exit('Splt must add to 1')
  
  num_rois = len(df.index.unique(level='roi'))

  tr_ind = int(ceil(num_rois*splt[0]))
  ts_ind = int(ceil(num_rois*splt[1])) + tr_ind

  ar = np.arange(num_rois)
  np.random.shuffle(ar)
  ar = ar+1 # roi id's start from 1

  train = ar[:tr_ind]
  test = ar[tr_ind:ts_ind]
  val = ar[ts_ind:]

  for i in train:
    df.loc[pd.IndexSlice[:, :, i],'training'] = 'train'

  for i in test:
    df.loc[pd.IndexSlice[:, :, i],'training'] = 'test'

  for i in val:
    df.loc[pd.IndexSlice[:, :, i],'training'] = 'validate'

  return(df)

In [8]:
# df_temp = trainTestSplit(raw_df,splt=[.5,.3,.3]) # should give error
df_temp = trainTestSplit(raw_df,splt=[.5,.3,.2])
df_temp.sample(15)

,,,,filename,resp,stim1,stim2,stim3,training
stack,label,roi,frame,,,,,,
5,tm2,14,2025,210815_0_5_stackRaw_mc_mix1_syt_result_2022021...,-0.075716,0.234538,0.000000,0.000000,train
4,t5,12,144,210815_0_4_stackRaw_mc_mix1_syt_result_2022021...,0.010364,-0.283397,0.000000,0.000000,train
9,tm2,5,1284,210815_0_9_stackRaw_mc_mix1_syt_result_2022021...,0.861132,-0.450496,-153.667218,0.004537,train
10,tm2,4,2049,210815_0_10_stackRaw_mc_mix1_syt_result_202202...,1.504061,-0.539944,11.222951,0.004290,train
8,tm4,9,2728,210815_0_8_stackRaw_mc_mix1_syt_result_2022021...,0.371231,0.182262,95.484980,0.030910,train
6,t5,12,398,210815_0_6_stackRaw_mc_mix1_syt_result_2022021...,0.020359,0.089130,66.419748,0.128629,train
8,tm4,15,4434,210815_0_8_stackRaw_mc_mix1_syt_result_2022021...,0.631193,0.252677,0.000000,0.000000,test
6,tm2,10,3563,210815_0_6_stackRaw_mc_mix1_syt_result_2022021...,0.403367,-0.414554,0.000000,0.000000,validate
5,tm9,6,797,210815_0_5_stackRaw_mc_mix1_syt_result_2022021...,-0.044935,-0.968656,10.488075,0.013048,test


## Combine Frames into Temporal Chunks

In [9]:
# select frame size
temporal_period_length = 200

In [20]:

def groupFrames(df, period):
  # inputs are data frame and temporal period length
  # df needs to be sorted correctly, the indexing should take care of that

  # check that all rois are the same size
  # if this is a problem we can write a more computationally heavy workthrough
  if (raw_df.groupby(['stack','roi']).count()['filename'].max() != raw_df.groupby(['stack','roi']).count()['filename'].min()):
    sys.exit('ROIs need to be the same length')

  num_stacks = len(df.index.unique(level='stack'))
  num_roi = len(df.index.unique(level='roi'))

  # get first label
  lbl_1 = raw_df.head(1).index.values[0][1]
  # use it to find the number of frames in a roi
  num_frames = raw_df.loc[(1,lbl_1,1),'filename'].count()
  # how many groups will ther be in each roi?
  num_pds = int(floor(num_frames/period))
  # how many left over
  remainder_pds = num_frames%period

  # iterate a column for one roi
  lst = [[li + 1] * period for li in range(0,num_pds)]
  lst = [li for sublist in lst for li in sublist] # https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-a-list-of-lists
  lst = lst + [lst[-1] + 1] * remainder_pds
  lst

  new_col = np.array(lst * num_stacks * num_roi)
  
  df['frame_group'] = new_col

  return(df)

In [22]:
# df_temp2 = groupFrames(raw_df,temporal_period_length)
# df_temp2

In [23]:
from operator import setitem
def stimResponse(df, stim, response, zero_point = 0):
  """
  Takes a df, the name of the col holding the stimulus, the name of the col holding the response, and the value to use as the zero point

  Returns the df with the relationship type and the relation value added

  A different function will calculate the relationship between time periods
  """


  # takes a stim column and response column and returns two new feature column
  # PR is Positive Response
  # NR is Negative Response
  # PS is Positive Stimulus
  # NS is Negative Stimulus
  df_in = df.copy()
  #
  PR = df_in[response] > zero_point
  NR = df_in[response] <= zero_point
  PS = df_in[stim] > zero_point
  NS = df_in[stim] <= zero_point

  PR = PR.rename('PR')
  NR = NR.rename('NR')
  PS = PS.rename('PS')
  NS = NS.rename('NS')

  # combine into df
  df = pd.concat([df_in[stim], df_in[response], PR,NR,PS,NS], axis=1)
  #set the data types back to boolean
  df = df.astype({'PR': bool,
                'PS': bool,
                'NR': bool,
                'PR': bool})

  # get min and max values for normalizing
  S_min = df[stim].min()
  S_max = df[stim].max()
  R_min = df[response].min()
  R_max = df[response].max()

  #add a column normalizing the stim and response
  df['stim_norm'] = df[stim].apply(lambda x: x/S_max if x>0 else x/S_min)
  df['resp_norm'] = df[response].apply(lambda x: x/R_max if x>0 else x/R_min)


  # multiply stim and resp
  df['relation'] = df['stim_norm'] * df['resp_norm']

  #Set a new column with the category each row's relationship falls under
  df.loc[(df['PR'] & df['PS']),'relation_type'] = 'PRPS'
  df.loc[(df['NR'] & df['NS']),'relation_type'] = 'NRNS'
  df.loc[(df['PR'] & df['NS']),'relation_type'] = 'PRNS'
  df.loc[(df['NR'] & df['PS']),'relation_type'] = 'NRPS'

  #UNCOMMENT this to create the len-4 matrix for the current time period directly

  # #one-hot encode the four categories
  # df = pd.get_dummies(df, columns=['relation_type'], prefix = stim)

  # #get a list of the columns created
  # dummy_cols = [col for col in df.columns if 'relation_type_' in col]

  # #multiply the dummy columns by the relation value to distribute the value to the appropriate column
  # for col in dummy_cols:
  #   df[col] = df[col] * df['relation']
  
  df_in['relation_type_' + stim] = df['relation_type']
  df_in['relation_' + stim] = df['relation']
  return(df_in)

In [24]:
#pass in the raw df, the name of the stimulus we want to use, the response column, and the zero-value (in this case zero)
df_temp_3 = stimResponse(raw_df, 'stim1', 'resp', 0)

In [25]:
df_temp_3.head()

filename  \
stack label roi frame                                                      
1     tm2   1   0      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...   
                1      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...   
                2      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...   
                3      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...   
                4      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...   

                           resp     stim1       stim2     stim3 training  \
stack label roi frame                                                      
1     tm2   1   0     -0.106575 -0.395877  160.447960  0.110063     test   
                1      0.244535 -0.395877    0.000000  0.000000     test   
                2      0.398458 -0.395877    0.000000  0.000000     test   
                3      0.619290 -0.138699  170.252676  0.030790     test   
                4      0.627615  0.890013  -62.485104  0.122134     test   

                      relation_type_stim1  relation_stim1  
stack label roi frame                                      
1     tm2   1   0                    NRNS        0.044529  
                1                    PRNS        0.023493  
                2                    PRNS        0.038280  
                3                    PRNS        0.020845  
                4                    PRPS        0.134750

In [26]:
def get_future_stim_and_relationship(df_in, stim, time_window): 
  """
  takes a df, the stimulus name we're comparing, and the time window we're looking to in the future (in frames)

  returns a new df with two extra columns containing the relationship type and the value at the future time period
  """

  #make copy of the df to avoid editing the one in memory
  df = df_in.copy()

  #procedurally generate the column names we need to reference
  current_relation_colname = 'relation_'+stim
  current_relation_type_colname = 'relation_type_'+stim

  #and the column names we're creating
  future_relation_colname = 'relation_'+stim+'_+'+str(time_window)
  future_relation_type_colname = 'relation_type_'+stim+'_+'+str(time_window)

  #reset the index so that we can join the new columns later
  df = df.reset_index()


  # for each of the new columns, do four steps

  #get the existing array of relation values or relation types

  #cut off the first x values, where x is the time window we want to look in the future (done by indexing [time_window:])
        #this will drop the time x values in the future to line up with the current time

  #pad the end of the array with zeros by the same number of values so that the lengths still match (done by concatenating with np.zeros(time_window))

  #set the new array as the new 'future' column
  df[future_relation_colname] = pd.concat([df[current_relation_colname] \
                                           .reset_index(drop=True)[time_window:], \
                                           pd.Series((np.zeros(time_window)))], ignore_index = True)
  
  #repeat for the relation type
  df[future_relation_type_colname] = pd.concat([df[current_relation_type_colname] \
                                           .reset_index(drop=True)[time_window:], \
                                           pd.Series((np.zeros(time_window)))], ignore_index = True)

  #need to stop the end of one stack from referencing the beginning of another

  #get the index of any row where the frame # is high enough that x-frames in the future would run into another stack
  #by checking that the frame number is greater than the max frame number - (time window-1) (corrects for index number)
  end_of_stack_index = df.loc[df['frame'] >= max(df['frame'])-(time_window-1)].index

  #use .loc to set the two new columns to None for those rows
  df.loc[end_of_stack_index, future_relation_colname] = None
  df.loc[end_of_stack_index, future_relation_type_colname] = None

  #set the index back to the way it was
  df = df.set_index(['stack','label','roi','frame'])

  return df

In [28]:
#check that the values match the records 10 in the future and that the last 10 rows in each stack are null for future values
df = get_future_stim_and_relationship(df_temp_3, 'stim1', 10)
df.tail(30)

filename  \
stack label roi frame                                                      
10    tm4   15  5483   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5484   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5485   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5486   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5487   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5488   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5489   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5490   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5491   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5492   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5493   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5494   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5495   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5496   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5497   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5498   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5499   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5500   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5501   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5502   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5503   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5504   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5505   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5506   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5507   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5508   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5509   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5510   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5511   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   
                5512   210815_0_10_stackRaw_mc_mix1_syt_result_202202...   

                           resp     stim1       stim2         stim3 training  \
stack label roi frame                                                          
10    tm4   15  5483   0.410775 -0.928950    0.000000  0.000000e+00     test   
                5484   0.597176 -0.928950    0.000000  0.000000e+00     test   
                5485   0.691033 -0.385567  127.344289  4.152338e-02     test   
                5486   0.783472  0.157816  102.290206  5.034420e-02     test   
                5487   0.681700  0.157816    0.000000  0.000000e+00     test   
                5488   0.645437  0.157816    0.000000  0.000000e+00     test   
                5489   0.621711 -0.099867  -62.608303  8.519235e-03     test   
                5490   0.477412 -0.357550   66.361290  7.474700e-03     test   
                5491   0.302191 -0.357550   14.624876  1.330373e-04     test   
                5492   0.278669 -0.345466  -12.361882  1.833777e-03     test   
                5493   0.344372 -0.009309  102.611405  4.637030e-02     test   
                5494   0.429321  0.336574   92.498896  1.199894e-01     test   
                5495   0.279488  0.457204   89.014950  5.814974e-02     test   
                5496   0.233815  0.461905   73.168936  1.098554e-02     test   
                5497  -0.018198 -0.200475 -122.457020  9.197023e-02     test   
                5498   0.072787 -0.531665 -140.057655  3.599874e-02     test   
                5499   0.021619 -0.531665   81.091849  9.244374e-08     test   
                5500   0.240001 -0.531665    0.000000  0.000000e+00     test   
                5501   0.191335

In [29]:
#with this approach, we need to drop the last x frames from each stack, so dropna
df = df.dropna()

In [30]:
#make a new column with the combo of current relation type and future relation type
df['full_relation_type'] = df['relation_type_stim1'] + '-' + df['relation_type_stim1_+10']
df.head()

filename  \
stack label roi frame                                                      
1     tm2   1   0      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...   
                1      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...   
                2      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...   
                3      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...   
                4      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...   

                           resp     stim1       stim2     stim3 training  \
stack label roi frame                                                      
1     tm2   1   0     -0.106575 -0.395877  160.447960  0.110063     test   
                1      0.244535 -0.395877    0.000000  0.000000     test   
                2      0.398458 -0.395877    0.000000  0.000000     test   
                3      0.619290 -0.138699  170.252676  0.030790     test   
                4      0.627615  0.890013  -62.485104  0.122134     test   

                      relation_type_stim1  relation_stim1  relation_stim1_+10  \
stack label roi frame                                                           
1     tm2   1   0                    NRNS        0.044529            0.000568   
                1                    PRNS        0.023493            0.035698   
                2                    PRNS        0.038280            0.128194   
                3                    PRNS        0.020845            0.096379   
                4                    PRPS        0.134750            0.087835   

                      relation_type_stim1_+10 full_relation_type  
stack label roi frame                                             
1     tm2   1   0                        PRNS          NRNS-PRNS  
                1                        PRPS          PRNS-PRPS  
                2                        PRPS          PRNS-PRPS  
                3                        PRPS          PRNS-PRPS  
                4                        PRPS          PRPS-PRPS

In [31]:
#one-hot encode the overall relationship, then multiply the new columns by the product of the relation values

df = pd.get_dummies(df, columns=['full_relation_type'], prefix = 'relation_stim1')

#get a list of the columns created
dummy_cols = [col for col in df.columns if 'relation_stim1' in col]

#multiply the dummy columns by the relation value to distribute the value to the appropriate column
for col in dummy_cols:
  df[col] = df[col] * (df['relation_stim1']*df['relation_stim1_+10'])

df.head(10) 

filename  \
stack label roi frame                                                      
1     tm2   1   0      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...   
                1      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...   
                2      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...   
                3      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...   
                4      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...   
                5      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...   
                6      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...   
                7      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...   
                8      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...   
                9      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...   

                           resp     stim1       stim2     stim3 training  \
stack label roi frame                                                      
1     tm2   1   0     -0.106575 -0.395877  160.447960  0.110063     test   
                1      0.244535 -0.395877    0.000000  0.000000     test   
                2      0.398458 -0.395877    0.000000  0.000000     test   
                3      0.619290 -0.138699  170.252676  0.030790     test   
                4      0.627615  0.890013  -62.485104  0.122134     test   
                5      0.611509  0.890013    0.000000  0.000000     test   
                6      0.365876  0.890013    0.000000  0.000000     test   
                7      0.289782  0.591530 -160.555209  0.018521     test   
                8      0.249789 -0.005437  -98.573619  0.135386     test   
                9      0.222600 -0.005437    0.000000  0.000000     test   

                      relation_type_stim1  relation_stim1  relation_stim1_+10  \
stack label roi frame                                                           
1     tm2   1   0                    NRNS    1.125795e-06        3.629198e-13   
                1                    PRNS    1.970170e-05        2.510677e-08   
                2                    PRNS    1.878503e-04        3.087067e-06   
                3                    PRNS    4.187694e-05        3.889907e-07   
                4                    PRPS    1.594875e-03        1.230450e-05   
                5                    PRPS    1.053308e-03        3.932878e-06   
                6                    PRPS    3.900491e-04        1.558399e-06   
                7                    PRPS    4.959005e-05        4.170929e-08   
                8                    PRNS    3.625492e-09        4.037647e-12   
                9                    PRNS    9.942141e-10        1.320260e-13   

                      relation_type_stim1_+10  ...  relation_stim1_NRPS-PRNS  \
stack label roi frame                          ...                             
1     tm2   1   0                        PRNS  ...                       0.0   
                1                        PRPS  ...                       0.0   
                2                        PRPS  ...                       0.0   
                3                        PRPS  ...                       0.0   
                4                        PRPS  ...                       0.0   
                5                        PRPS  ...                       0.0   
                6                        PRPS  ...                       0.0   
                7                        PRPS  ...                       0.0   
                8                        PRPS  ...                       0.0   
                9                        PRPS  ...                       0.0   

                       relation_stim1_NRPS-PRPS  relation_stim1_PRNS-NRNS  \
stack label roi frame                                                       
1     tm2   1   0                           0.0                       0.0   
                1                           0.0                       0.0

In [32]:
# df.to_csv('drive/MyDrive/DS6011_Capstone_VisualNeuroscience/Seeded_CNMF/Extracted_Features/210815_0_20220213T070259_sparse_stim1_draft.csv')